In [ ]:
import numpy as np
import torch
from functions import load_test_data

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
#3flavor_clean, 3flavor_poisson, nsi_clean, nsi_poisson
learn_target = '3flavor_poisson'

data_all = load_test_data(learn_target)

In [ ]:
index = 1
flow = torch.load('./nf/{}.pt'.format(index))

In [ ]:
n_sample = 1000000
IO_samples = flow.sample(num_samples=n_sample,
            context=torch.tensor(np.array([data_all[1]/1000]), dtype=torch.float32).to(device)).cpu().detach().numpy().reshape(-1, 4)
NO_samples = flow.sample(num_samples=n_sample,
            context=torch.tensor(np.array([data_all[0]/1000]), dtype=torch.float32).to(device)).cpu().detach().numpy().reshape(-1, 4)


In [ ]:
np.savez("./contour_IO/nf.npz", IO_samples)
np.savez("./contour_NO/nf.npz", NO_samples)